In [3]:
from data.odds_api import OddsAPIClient
from core.vig import strip_vig_outrights
from core.strength import calc_team_strength
from core.tournament import simulate_many

raw = OddsAPIClient().to_dataframe(OddsAPIClient().fetch())
df = strip_vig_outrights(raw)
strength_df = calc_team_strength(df)


In [8]:
def run_mc(n_runs: int = 400):
    simulate_many(strength_df, n_runs=n_runs, seed=0)

In [9]:
import cProfile, pathlib, datetime as dt
from datetime import timezone

stamp = dt.datetime.now(tz=timezone.utc).strftime("%Y%m%dT%H%M%SZ")
out   = pathlib.Path.cwd() / f"profile_{stamp}.prof"

cProfile.run("run_mc()", out.as_posix())    

print(f"Wrote {out.name} – view with:\n  snakeviz notebooks/{out.name}")


Wrote profile_20250604T230752Z.prof – view with:
  snakeviz notebooks/profile_20250604T230752Z.prof


In [7]:
import inspect, textwrap, core.tournament as T
print(textwrap.dedent(inspect.getsource(T.simulate_many))[:400])


def simulate_many(strength_df: pd.DataFrame, n_runs: int = 100_000, seed: int | None = None) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    winners = [simulate_tournament_once(strength_df, rng) for _ in range(n_runs)]
    vc = pd.Series(winners).value_counts().rename_axis("team").to_frame("champion_prob")
    vc["champion_prob"] /= n_runs
    return vc.reset_index()



In [2]:
import sys, pprint, importlib.util
pprint.pprint(sys.path[:5])               # should show '/path/to/world-cup-sim'

spec = importlib.util.find_spec("src")
print("spec is", spec)


['/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/Users/abrahmazulay/Dev/world-cup-sim/.venv/lib/python3.10/site-packages']
spec is None
